<a href="https://colab.research.google.com/github/Malvika55/text_to_video/blob/main/text_to__video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe=DiffusionPipeline.from_pretrained("ali-vilab/text-to-video-ms-1.7b",torch_dtype=torch.float16,variant="fp16")
pipe.scheduler=DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()



In [ ]:
prompt = "spiderman is surfing"
video_frames=pipe(prompt,num_inference_steps=25,).frames

# The output from `pipe().frames` is typically a torch.Tensor of shape
# `(batch_size, num_frames, num_channels, height, width)`.
# When assigned, it often becomes a numpy array of this shape (e.g., (1, 25, 3, 256, 256) for a single prompt).
# The `export_to_video` function in diffusers expects a 4D array or a list of images.
# It does not handle a 5D array (batch dimension).
# We need to remove the batch dimension if it exists.
if isinstance(video_frames, torch.Tensor):
    video_frames = video_frames.cpu().numpy()

if video_frames.ndim == 5 and video_frames.shape[0] == 1:
    video_frames = video_frames.squeeze(0) # Reduce from (1, N, C, H, W) to (N, C, H, W)

# Now, `video_frames` is (num_frames, channels, height, width) which `export_to_video` can handle.
video_path = export_to_video(video_frames)
video_name= video_path.replace('/tmp','')
print ('Name:',video_name)
torch.cuda.empty_cache()

# Task
Clone the content from `https://github.com/Malvika55/text_to_video` into the current Colab environment.

## Clone Source Repository

### Subtask:
Clone the content from `https://github.com/Malvika55/text_to_video` into the current Colab environment. This will allow you to access all the files locally within Colab.


**Reasoning**:
To clone the specified GitHub repository, I will use the `!git clone` command in a code block, as instructed.



In [ ]:
import os

!git clone https://github.com/Malvika55/text_to_video

# List the contents of the cloned directory to verify
print(os.listdir('text_to_video'))

## Final Task

### Subtask:
After the repository is cloned to Colab, you will need to manually create your target repository on GitHub and then upload the files from the Colab environment to your new GitHub repository.


## Summary:

### Data Analysis Key Findings
*   The `git clone` command successfully downloaded the `text_to_video` repository from GitHub into the Colab environment.
*   Verification using `os.listdir()` confirmed that the directory `text_to_video` was created and contained files, specifically `tmpwq59_yxb (1).mp4` and `.git`.

### Insights or Next Steps
*   The content of the `text_to_video` repository is now accessible within the Colab environment.
*   The next step will involve manually creating a new GitHub repository and then uploading the cloned files from Colab to that new GitHub repository.
